In [2]:
# coding:utf-8

import sys
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import jieba
from jieba import analyse
import numpy as np
from sklearn import metrics
from sklearn.svm import SVC
import random


def get_segment(all=False):
    csvfile = open('movie.csv', newline='')
    rows = csv.DictReader(csvfile)
    temps=[]
    for row in rows:
        # 對標題和內容切詞
        seg_list = jieba.cut(row['名稱'] + " " + row['分類'] + " " + row['劇情介紹'],cut_all=False)
        line = ""
        for stri in seg_list:
            line = line + " " + stri
        line = line.replace('\'', '')
        temp={}
        temp['名稱'] = row['名稱']
        temp['分類'] = row['分類']
        temp['內容'] = line
        temps.append(temp)
    labels = ['名稱', '分類', '內容']
    with open('movie1.csv', 'w', encoding='utf-8-sig') as f:
      writer = csv.DictWriter(f, fieldnames=labels)
      writer.writeheader()
      for i in temps:
        writer.writerow(i)

def product_category():
  category={}
  category[1] = '動作'
  category[2] = '冒險'
  category[3] = '科幻'
  category[4] = '奇幻'
  category[5] = '劇情'
  category[6] = '犯罪'
  category[7] = '恐怖'
  category[8] = '懸疑/驚悚'
  category[9] = '喜劇'
  category[10] = '愛情'
  category[11] = '溫馨/家庭'
  category[12] = '動畫'
  category[13] = '戰爭'
  category[14] = '音樂/歌舞'
  category[15] = '歷史/傳記'
  category[16] = '紀錄片'
  category[17] = '勵志'
  category[18] = '武俠'
  category[19] = '影展'
  category[20] = '戲劇'
  category[21] = '影集'
  return category

get_segment()
category_ids = []
category = product_category()
test_typs = []
test_titles = []
rnds = []
test = []
csvfile = open('movie1.csv', newline='', encoding='utf-8-sig')
rows = csv.DictReader(csvfile)
line = ""
s = list(rows)
k = 0
while k < 500:
    rnd = random.randint(0,len(s)-1)
    if rnd not in rnds:
      title = dict(s[rnd])['名稱']
      test_titles.append(title)
      typ = dict(s[rnd])['分類']
      test_typs.append(typ)
      segment = dict(s[rnd])['內容']
      test.append(segment)
      rnds.append(rnd)
      k = k + 1

train_typs = []
train_titles = []
train = []
csvfile = open('movie1.csv', newline='', encoding='utf-8-sig')
rows = csv.DictReader(csvfile)
line = ""
for i in range(0,len(s)-1):
  if i not in rnds:
      title = dict(s[i])['名稱']
      train_titles.append(title)
      typ = dict(s[i])['分類']
      train_typs.append(typ)
      segment = dict(s[i])['內容']
      train.append(segment)
      for j in range(1,22):
                if dict(s[i])['分類']==category[j]:
                  category_ids.append(j)


def classification(train, test, category_ids):
  vectorizer=CountVectorizer()
  train_mat = vectorizer.fit_transform(train)
  test_mat = vectorizer.transform(test)
  transformer=TfidfTransformer()
  train_tfidf=transformer.fit_transform(train_mat)
  test_tfidf=transformer.transform(test_mat)
  y = np.array(category_ids)
  model = SVC()
  model.fit(train_tfidf, y)
  predicted = model.predict(test_tfidf)
  return predicted

predicted = classification(train, test, category_ids)

def print_out(predicted, category, test_titles, test_typs):
  correct = 0
  for index in range(0, len(test_titles)):
        predicted_category = category[predicted[index]]
        print("predict title: %s <==============> predicted category: %s  true category: %s" % (test_titles[index], predicted_category, test_typs[index]))
        if predicted_category==test_typs[index]:
          correct += 1
  print("本次訓練的正確性為：%s" % (correct/500))

print_out(predicted, category, test_titles, test_typs)

predict title: 我的海星爸爸 <==============> predicted category: 劇情  true category: 劇情
predict title: 添好孕 <==============> predicted category: 劇情  true category: 劇情
predict title: 我的青春煉獄 <==============> predicted category: 劇情  true category: 劇情
predict title: 鋼鐵墳墓 <==============> predicted category: 動作  true category: 動作
predict title: 獵日風暴 <==============> predicted category: 動作  true category: 動作
predict title: 沙丘 <==============> predicted category: 動作  true category: 動作
predict title: 我很好 <==============> predicted category: 劇情  true category: 劇情
predict title: 下女的誘惑 <==============> predicted category: 劇情  true category: 劇情
predict title: 甜蜜的永遠 <==============> predicted category: 劇情  true category: 劇情
predict title: 與海豚共舞 <==============> predicted category: 劇情  true category: 紀錄片
predict title: 青春電幻物語：數位修復版 <==============> predicted category: 劇情  true category: 劇情
predict title: 巡弋狙擊手 <==============> predicted category: 劇情  true category: 動作
predict title: 絕命救援 <==============> pr